# AirbyteLoader

>[Airbyte](https://github.com/airbytehq/airbyte)是一个用于从API、数据库和文件到数据仓库和数据湖的ELT管道的数据集成平台。它拥有最大的ELT连接器目录，可连接到数据仓库和数据库。

本文介绍如何将Airbyte中的任何数据源加载到LangChain文档中

## 安装

要使用`AirbyteLoader`，您需要安装`langchain-airbyte`集成包。

In [1]:
%pip install -qU langchain-airbyte

注意：目前，`airbyte`库不支持Pydantic v2。
请降级到Pydantic v1以使用此包。

注意：此包目前还要求Python 3.10+。

## 加载文档

默认情况下，`AirbyteLoader`将从流中加载任何结构化数据并输出yaml格式的文档。

In [6]:
from langchain_airbyte import AirbyteLoader

loader = AirbyteLoader(
    source="source-faker",
    stream="users",
    config={"count": 10},
)
docs = loader.load()
print(docs[0].page_content[:500])

```yaml
academic_degree: PhD
address:
  city: Lauderdale Lakes
  country_code: FI
  postal_code: '75466'
  province: New Jersey
  state: Hawaii
  street_name: Stoneyford
  street_number: '1112'
age: 44
blood_type: "O\u2212"
created_at: '2004-04-02T13:05:27+00:00'
email: bread2099+1@outlook.com
gender: Fluid
height: '1.62'
id: 1
language: Belarusian
name: Moses
nationality: Dutch
occupation: Track Worker
telephone: 1-467-194-2318
title: M.Sc.Tech.
updated_at: '2024-02-27T16:41:01+00:00'
weight: 6


您还可以指定用于格式化文档的自定义提示模板：

In [7]:
from langchain_core.prompts import PromptTemplate

loader_templated = AirbyteLoader(
    source="source-faker",
    stream="users",
    config={"count": 10},
    template=PromptTemplate.from_template(
        "My name is {name} and I am {height} meters tall."
    ),
)
docs_templated = loader_templated.load()
print(docs_templated[0].page_content)

My name is Verdie and I am 1.73 meters tall.


## 懒加载文档

`AirbyteLoader`的强大功能之一是能够从上游源加载大型文档。处理大型数据集时，默认的`.load()`行为可能会很慢且内存密集。为避免这种情况，您可以使用`.lazy_load()`方法以更节省内存的方式加载文档。

In [11]:
import time

loader = AirbyteLoader(
    source="source-faker",
    stream="users",
    config={"count": 3},
    template=PromptTemplate.from_template(
        "My name is {name} and I am {height} meters tall."
    ),
)

start_time = time.time()
my_iterator = loader.lazy_load()
print(
    f"Just calling lazy load is quick! This took {time.time() - start_time:.4f} seconds"
)

Just calling lazy load is quick! This took 0.0001 seconds


您可以在文档生成时对其进行迭代：

In [12]:
for doc in my_iterator:
    print(doc.page_content)

My name is Andera and I am 1.91 meters tall.
My name is Jody and I am 1.85 meters tall.
My name is Zonia and I am 1.53 meters tall.


您还可以通过`.alazy_load()`以异步方式懒加载文档：

In [13]:
loader = AirbyteLoader(
    source="source-faker",
    stream="users",
    config={"count": 3},
    template=PromptTemplate.from_template(
        "My name is {name} and I am {height} meters tall."
    ),
)

my_async_iterator = loader.alazy_load()

async for doc in my_async_iterator:
    print(doc.page_content)

My name is Carmelina and I am 1.74 meters tall.
My name is Ali and I am 1.90 meters tall.
My name is Rochell and I am 1.83 meters tall.


## 配置

`AirbyteLoader`可以通过以下选项进行配置：

- `source`（str，必填）：要从中加载的Airbyte数据源名称。
- `stream`（str，必填）：要从中加载的流名称（Airbyte数据源可以返回多个流）
- `config`（dict，必填）：Airbyte数据源的配置
- `template`（PromptTemplate，可选）：用于格式化文档的自定义提示模板
- `include_metadata`（bool，可选，默认为True）：是否在输出文档中包含所有字段作为元数据

配置的大部分内容将在`config`中，您可以在[Airbyte文档](https://docs.airbyte.com/integrations/)中每个源的"Config field reference"中找到特定的配置选项。